# Scheduling Model #1

## Overall Concept

The decision variables $P_{i,j}$, the number of practice units for item $i$ during the
time slot $j$ (e.g., a day in a sequence of days), are organized into a tableau like the following:

|          | $1$   | $2$ |$\ldots$|$m$|
|:--------:|:-------:|:-------:|:--------:|:--------:|
|$item_1$  | $P_{1,1}$    | $P_{1,2}$   |   |   $P_{1,m}$      |
|$item_2$  | $P_{2,1}$    | $P_{2,2}$   |  |    $P_{2,1}$     |
|$\vdots$  |          |   |    $\ddots$     |       |
|$item_{n}$| $P_{n,1}$|$P_{n,2}$   |        | $P_{n,m}$       |

where $n =nitems$, and $m = nslots$.

Rows are the items to be studied and columns are time 
slots (e.g., days). Constraints are expressed in terms of sums over
rows and columns, or in the 
case of the density constraint, over a series of adjacent columns.

## LP Formulation

### Counts and Indices

\begin{align*}
&nitems             && \text{Number of items to schedule} \\
&nslots             && \text{Number of time slots available} \\
&i \in \{0, 1,  \ldots, nitems-1 \} && \text{Index over items}\\
&t \in \{0, 1, \ldots, nslots-1 \} && \text{Index over time slots} \\
&t_\delta \in \{0, 1, \ldots, winsz-1\} && \text{Index over time window}
\end{align*}

See below for the use of $t_\delta$ and $winsz$ to express 
limits on practice "density".

### Data

\begin{align*}
&ergused_i          && \text{Amount of mental "energy" required per unit of practice time for task i ("ergs" of energy)} \\
&ergavail_t        && \text{Amount of mental "energy" available per day ("ergs"/day)}\\
&timeavail_t       && \text{Amount of time resource available during a training slot $t$ (time units per day)} \\
&timeper_i         && \text{Practice time needed to master item $i$} \\
&minperwin_i  && \text{Minimum number of training slots for item $i$ within a window (days)} \\
&maxperwin_i  && \text{Maximum number of training slots for item $i$ within a window (days)} \\
&winsz            && \text{Used to express minimum and maximum "density" of practice days (days)}\\
\end{align*}

### Variables

\begin{align*}
&P_{i,t}      && \text{Amount of practice in time slot $t$ for item $i$ (number of units of time study)}\\
&ERG_{t}           && \text{Energy expended on a given day (macro over $P$)}\\
&ERGD_t     && \text{Amount by which energy expenditure decreased at $t$ from $t-1$}\\
&ERGI_t     && \text{Amount by which energy expenditure increased at $t$ from $t-1$}
\end{align*}

where $P_{i,t} \in \mathbb Z_{\ge 0};\ ERG_{t}, ERGI_{t}, ERGD_{t}\in \mathbb R_{\ne 0}$.


### Objective and Constraints

Minimize

$$
U = \underbrace{\sum_{i, t} P_{i,t} - \sum_{i}{timeper_i} }_{\text{(O1) Excess time spent}} 
        + \underbrace{\sum_{t=1}{(ERGI_t - ERGD_t)}}_{\text{(O2) Energy smoothing}}
$$

s.t.

\begin{align*}
&& \sum_{i}{P_{i,t}} &= timeavail_t\ \forall\ t 
  && \text{(C1) Use all available time, and no more} \\
&& \sum_{t}{P_{i,t}} &\geq timeper_i\   \forall\ i 
  && \text{(C2) Practice time greater than required to learn item} \\
&& \sum_{i}{P_{i,t + t_\delta}}  &\geq minperwin\ \forall i, t \forall t_\delta 
  && \text{(C3) Practice spacing - min. amount in window} \\
&& \sum_{i}{P_{i,t + t_\delta}}  &\leq maxperwin\ \forall i, t
  && \text{(C4) Practice spacing for rest - max. amount in window}\\
&& ERG_{t} &= \sum_i{ergused_i \times P_{i,t}}\ \forall t
  && \text{(C5) Defn. Used below.}\\
&& ERGI_{t+1} - ERGD_{t+1} &= ERG_{t+1} - ERG_{t}\ \forall t
  && \text{(C6) Defn. Change in either direction. One always zero.}\\
\end{align*}


### Discussion

The objective function is composed of three positive terms
to be minimized. The first term (O1)
captures the amount the sum of the total amount of practice time across all
items. The second term (O2) is the relative energy expenditure on practice
from day to day. This smooths energy expenditure, which has the effect
of preferring spreading around the more energy-sapping work, and increasing
the diversity with respect to intensity on any given day. I've included
the constant term so that the objective values can be compared across different
length schedules. 

The first two constraints are essentially supply and demand constraints. 
The time spent practicing must be equal to the time available (C1)
and each item requires that a minimum amount of practice is done to master the
item (C2). Note that, I could have relaxed these constraints and used a model
similar to a transportation problem, incorporating dummy supply or demand. Instead,
I assume that a shortage of supply will result an infeasible schedule, forcing
a user to revisit the amount of time being allocated overall or per day. 


The constraints (C3) and (C4) are contraints that limit the number of consecutive
days spent practicing. They are expressed in terms of a time window
parameter. The goal is to ensure an adequate number of days spent resting while
ensuring that practice is resumed before too long passes.

The constraints (C5) and C(6) relate to energy expenditure. The equality
constraints define the difference in energy expenditure from day to day. The
difference $ERGI_{t+1} - ERGD_{t+1}$ of two positive variables captures
this change. Either $ERGI_{t+1}$ or $ERGD_{t+1}$ will always be driven
to zero by the minimization.

Finally, we require that the amount of practice of a given item on a given
day $P_{i,t}$ is a non-negative *integer* to enforce the fixed-size
practice sessions, and that all other variables are non-negative 
real numbers.

There are several known issues with this formulation that I will need to address:

* Incommensurability of the three terms in the objective function. A common metric that captures utility with appropriate scaling factors is needed. This should correlate
with the expected "satisfaction" experience by the practicer. I'll need to add
coefficients to the required Data that encode this.

* There are no constraints related to energy except for the smoothing constraint. 
It is probably desirable to specify minimum and maximum expenditures in some way.

* There's no way to say that after the required practice is done for an item
that the rest interval increases. My current workaround is to imagine that
the user organizes his schedule for some period of time around reaching the
next plateau and then builds a new model based on the new levels reached. This
may not be ideal. I'm not sure.

### Diagnostic Metrics

The slacks and excesses are useful information for a user of the scheduling tool
if presented properly. In addition, some efficiency-related metrics are
probably going to be useful:

* Time efficiency. Actual time spent practicing an item $i$ versus what's required:
  
$$
R_i = \frac{\sum_t{P_{i,t}}}{timeper_i}
$$

* The most and least efficient tasks are then $arg\,min_i\ R_i$ and $arg\,imax_i\ R_i$

* Daily intensity is:

$$
I_t = \frac{ERG_{t}}{timeavail_t}
$$

The sums of these over all items may also be useful to present to the user.

## Examples

In [20]:
from models import *

### Example #1: 4 items, evaluating 3 possible time slot counts

This illustrates how in order to meet consecutive practice/rest constraints, it's necessary
to put in more practice for some items than is neeeded. This is one somewhat counter-intuitive
feature of the model. There's no notion of the task being "done". I don't know how to add this.

In [25]:
params = model_parameters(n_items=4, # 4 things to practice
           # Two time units per slot for (up to) 12 slts
           time_avail=[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
           # Five Units of energy available per slot
           erg_avail= [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], 
           # Practice time needed to complete each item
           time_per=[2, 2, 2, 2],
           # Energy needed                          
           erg_used=[1, 2, 1, 2],          
           # Names to display               
           item_names=["1-Easy", "2-Hard", "3-Easy", "4-Hard"],     
           win_sz=3, min_per_win=1, max_per_win=2)

In [26]:
for slot_count in [4, 7, 10]:
    solver = solve(**params, min_slots=slot_count, max_slots=slot_count)
    if solver.status == LpStatusOptimal:
        show_solution(solver)

Schedule starting tomorrow:
     name  04/03  04/04  04/05  04/06
0  1-Easy    0.0    0.0    1.0    1.0
1  2-Hard    0.0    1.0    0.0    1.0
2  3-Easy    1.0    1.0    0.0    0.0
3  4-Hard    1.0    0.0    1.0    0.0
item 0 re-practice ratio = 0.0
item 1 re-practice ratio = 0.0
item 2 re-practice ratio = 0.0
item 3 re-practice ratio = 0.0
Overall re-practice ratio is 0.0

Schedule starting tomorrow:
     name  04/03  04/04  04/05  04/06  04/07  04/08  04/09
0  1-Easy    1.0    0.0    0.0    1.0    0.0    0.0    0.0
1  2-Hard    1.0    1.0    0.0    1.0    0.0    0.0    0.0
2  3-Easy    0.0    1.0    1.0    0.0    1.0    0.0    0.0
3  4-Hard    0.0    0.0    1.0    0.0    1.0    0.0    0.0
item 0 re-practice ratio = 0.0
item 1 re-practice ratio = 0.5
item 2 re-practice ratio = 0.5
item 3 re-practice ratio = 0.0
Overall re-practice ratio is 0.25

Schedule starting tomorrow:
     name  04/03  04/04  04/05  04/06  04/07  04/08  04/09  04/10  04/11  04/12
0  1-Easy    0.0    1.0    0.0    